In [2]:
pip install schedule


Note: you may need to restart the kernel to use updated packages.


In [1]:
import schedule
import time
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Your model loading and training functions
def load_and_train_model():
    # Load the dataset (assumed to be updated regularly)
    df = pd.read_csv('loan_data.csv')  # Update path as necessary
    
    # Preprocessing steps (ensure you handle missing values and categorical encoding)
    df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)
    df = pd.get_dummies(df, drop_first=True)
    scaler = StandardScaler()
    df[['LoanAmount', 'Income']] = scaler.fit_transform(df[['LoanAmount', 'Income']])
    
    # Split the data
    X = df.drop(columns=['Default'])
    y = df['Default']
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Initialize and train model
    model = LogisticRegression(class_weight='balanced')
    model.fit(X_train, y_train)
    
    # Prediction and evaluation
    y_pred = model.predict(X_test)
    
    # Print evaluation metrics
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

    # Optionally save the model for future use (in case you want to avoid retraining)
    import joblib
    joblib.dump(model, 'loan_default_model.pkl')

# Schedule the batch process to run monthly (or any time interval)
def job():
    print("Running batch job for loan default prediction...")
    load_and_train_model()

# Schedule to run every 30 days
schedule.every(30).days.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)


KeyboardInterrupt: 